# Mettabook

## 1. Setup

### 1.1 Imports

In [10]:
import pandas as pd

# Enable auto-reload of modules
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from IPython.display import display
from mettabook_widgets import (
    JobLauncher,
    MetricsFetcher,
    ReplayViewer,
    TrainingConfigurator,
    WandBConnector,
)
from plotly.subplots import make_subplots

%matplotlib inline
plt.style.use("default")

print("Setup complete! Auto-reload enabled.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Setup complete! Auto-reload enabled.


### 1.2 Initialize Components

In [11]:
from run_store import get_runstore

run_store = get_runstore()

# Create widgets using the new stateless methods
config_widgets = TrainingConfigurator.create_widgets()
launcher_widgets = JobLauncher.create_widgets(config_widgets)
wandb_widgets = WandBConnector.create_widgets()
fetcher_widgets = MetricsFetcher.create_widgets()
replay_widgets = ReplayViewer.create_widgets(fetcher_widgets)  # Pass fetcher_widgets

## 2. Training

### 2.1 Configure and Launch Training Job

In [12]:
display(TrainingConfigurator.display(config_widgets))
display(JobLauncher.display(launcher_widgets))

### 2.2 View All Runs

Add runs you want to track with "Track Run"
Click "Refresh All" (and wait a minute) to get updated statuses on each job.

In [4]:
# Display RunStore table with all your runs
rs = get_runstore()
rs.to_widget()

## 3. Analyze Runs

You can analyze metrics from one or multiple runs. Copy run names from the RunStore table above.

### 3.1 Fetch Metrics from W&B

Enter one or more run names (one per line) to fetch their metrics. This will also automatically fetch available replay URLs:

In [5]:
display(MetricsFetcher.display(fetcher_widgets))

### 3.2 Analyze Metrics

The fetched metrics are stored in `fetcher_widgets['state']['metrics_dfs']` as a dictionary mapping run names to pandas DataFrames.

In [7]:
# Plot overview metrics for all fetched runs
metrics_dfs = fetcher_widgets["state"]["metrics_dfs"]
if not metrics_dfs:
    print("No metrics data available. Please fetch metrics first.")
else:
    print(f"Plotting metrics for {len(metrics_dfs)} runs")

    # Find common metrics across all runs
    all_columns = set()
    for _, df in metrics_dfs.items():
        all_columns.update(df.columns)

    # Filter for overview metrics
    include_prefixes = ["overview/"]
    plot_cols = []

    for col in all_columns:
        if not any(col.startswith(prefix) for prefix in include_prefixes):
            continue
        # Check if this column exists in at least one run with numeric data
        has_numeric_data = False
        for df in metrics_dfs.values():
            if col in df.columns and pd.api.types.is_numeric_dtype(df[col]) and df[col].nunique() > 1:
                has_numeric_data = True
                break
        if has_numeric_data:
            plot_cols.append(col)

    if not plot_cols:
        print("No plottable metrics found")
    else:
        # Calculate grid dimensions
        n_metrics = len(plot_cols)
        n_cols = min(3, n_metrics)  # Max 3 columns
        n_rows = (n_metrics + n_cols - 1) // n_cols

        # Create subplots
        fig = make_subplots(
            rows=n_rows,
            cols=n_cols,
            subplot_titles=[col.replace("overview/", "").replace("_", " ") for col in plot_cols],
            vertical_spacing=0.08,
            horizontal_spacing=0.1,
        )

        # Color palette for different runs
        colors = ["blue", "red", "green", "orange", "purple", "brown", "pink", "gray", "olive", "cyan"]

        # Add traces for each metric and each run
        for idx, col in enumerate(plot_cols):
            row = (idx // n_cols) + 1
            col_idx = (idx % n_cols) + 1

            # Plot each run for this metric
            for run_idx, (run_name, df) in enumerate(metrics_dfs.items()):
                if col in df.columns and "_step" in df.columns:
                    color = colors[run_idx % len(colors)]

                    # Only show legend on first subplot to avoid clutter
                    show_legend = idx == 0

                    fig.add_trace(
                        go.Scatter(
                            x=df["_step"],
                            y=df[col],
                            mode="lines",
                            name=run_name,
                            line=dict(color=color, width=2),
                            showlegend=show_legend,
                            legendgroup=run_name,  # Group all traces from same run
                        ),
                        row=row,
                        col=col_idx,
                    )

        # Update layout
        runs_text = "run" if len(metrics_dfs) == 1 else "runs"
        fig.update_layout(
            height=250 * n_rows,
            title_text=f"Overview Metrics ({len(metrics_dfs)} {runs_text})",
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        )

        # Update x-axes labels for bottom row
        for col_idx in range(1, min(n_cols, n_metrics) + 1):
            fig.update_xaxes(title_text="Steps", row=n_rows, col=col_idx)

        fig.show()

Plotting metrics for 1 runs


In [8]:
display(ReplayViewer.display(replay_widgets))

In [9]:
ReplayViewer.display_iframe(replay_widgets, width=1000, height=600)

Loading MettaScope viewer...

Direct link: https://metta-ai.github.io/metta/?replayUrl=https://softmax-public.s3.amazonaws.com/replays/daveey.lp.16x4.bptt8/89c52a86222e/9446add0-a298-4471-a55c-510f6ddbc9ba.json.z
